<div class="contentcontainer med left" style="margin-left: -50px;">
<dl class="dl-horizontal">
  <dt>Description</dt> <dd>Price Stream example</dd>
  <dt>Author</dt> <dd>Jean-Luc Stevens</dd>
  <dt>HoloViews</dt> <dd>>1.6.2</dd>
  <dt>Python</dt> <dd>1.7</dd>
</dl>
</div>

This short example demonstrates a simple example of how to capture the history of a HoloViews stream, which is a simple object with a set of parameters that allow events to affect a displayable HoloViews object. For more information on streams, please view the [quickstart guide](../quickstart/Streams.ipynb).

This notebook uses core HoloViews together with the matplotlib backend to demonstrate how you could visualize a timeseries by capturing live events, namely the price of some commodity. For this example, lets say we have a source of data (e.g online) updating us with the current price of sugar.

In [ ]:
import holoviews as hv
import numpy as np
from holoviews.streams import Stream
hv.notebook_extension()

To start we declare a custom Stream (see the [quickstart guide](../quickstart/Streams.ipynb) for another simple example) by subclassing from ``Stream`` and declaring a suitable parameter:

In [ ]:
import param

class SugarPrice(Stream):
    """
    The SugarPrice stream that has a single parameter, namely price.
    """
    price = param.Number(default=0, doc="""
        The current price of hypothetical sugar""")

As expected, we can make an instance of ``SugarPrice``, inspect the price of sugar and update it as necessary:

In [ ]:
sugar_price = SugarPrice()
print('Initial price of sugar: %r' % sugar_price.price)

In [ ]:
sugar_price.update(price=0.5)
print('Updated price of sugar: %r' % sugar_price.price)

The difference between this example and most other streams examples is that we will visualize the price of sugar over some history. As ``Stream`` objects do not automatically keep track of previous values, we have to do this ourselves. Thankfully, the Python standard library offers a useful ``deque`` object that can help us with this task:

In [ ]:
from collections import deque

def price_curve(price, history=deque(maxlen=100)):
    history.append(price)
    return hv.Curve(list(history), group='Simulated price of sugar')

This callback declare a ``deque`` to store a hundred sugar price values. Every time it is called, it add the current price to the ``deque`` and represents the full history as a ``hv.Curve`` object.

In [ ]:
hv.DynamicMap(price_curve, kdims=[], streams=[sugar_price])

At this point we can update the visualization with sugar prices that could come from any source (e.g a live stream online). For this example, we will just generate 150 values between zero and one. When you run the following cell, you'll see the visualization dynamically update above:

In [ ]:
for i in range(150):
    sugar_price.update(price=np.random.rand())

Using ``deque`` in this way is quick and convenient for building such a visualization. It is not necessarily the most robust way, as a single deque object has been declared as a default argument of our callback function. This makes it hard to reset the history and makes it difficult to use the same function to model multiple price streams. A more robust approach is to write a callable class while declaring more appropriate key and value dimension names:

In [ ]:
class PriceCurve(object):
    
    def __init__(self):
        self.history=deque(maxlen=100)
        
    def __call__(self, price):
        self.history.append(price)
        return hv.Curve(list(self.history),  kdims=['Count'], vdims=['Price'], 
                        group='Simulated price of sugar')

Using our class, we can visualize multiple independent price histories without a problem, by declaring separate ``deque`` and stream state. Here is a price stream visualized with our random numbers varying between zero and ten:

In [ ]:
sugar_price_10x = SugarPrice(price=10)
hv.DynamicMap(PriceCurve(), kdims=[], streams=[sugar_price_10x])

In [ ]:
for i in range(150):
    sugar_price_10x.update(price=10*np.random.rand())

And a price stream visualized with our random numbers varying between zero and a hundred:

In [ ]:
sugar_price_100x = SugarPrice(price=100)
hv.DynamicMap(PriceCurve(), kdims=[], streams=[sugar_price_100x])

In [ ]:
for i in range(150):
    sugar_price_100x.update(price=100*np.random.rand())

Of course, you could use any approach you like to store the history! This notebook simply illustrates two approaches, namely using ``deque`` in a simple function or using ``deque`` in a custom class. This simple example could also be extended in various ways: for instance, you might give the ``SugarPrice`` stream an additional parameter, namely a timestamp (e.g the value of ``time.time()``) which could the be plotted appropriately along the x-axis. Try it!